<a href="https://colab.research.google.com/github/Akhilchegonda/Akhil_INFO5731_Fall2023/blob/main/Akhil_ChegondaIn_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

In [1]:
# Previous Code
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import pandas as pd
Headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

titles = []
text=[]
for pg in range(1,4000):
  url = 'https://www.amazon.com/Konquest-KBP-2704A-Automatic-Pressure-Monitor/product-reviews/B077CXGSJW/ref=cm_cr_getr_d_paging_btm_next_7?ie=UTF8&reviewerType=all_reviews&pageNumber={'
  pg = requests.get(url, headers=Headers)
  s = BeautifulSoup(pg.content,'html.parser')
  reviews = s.find_all('a',class_='review-title-content')
  for i in range(len(reviews)):
    titles.append(reviews[i].get_text().strip())
  reviews= s.find_all('span',class_='review-text-content')
  for r in range(0,len(reviews)):
    text.append(reviews[r].get_text().strip())
df_reviews= pd.DataFrame({
    'Title' : titles,
    'Review': text
})

# Save the DataFrame as a CSV file
df_reviews.to_csv('reviews.csv', index=False)

# Read the saved CSV file and print the first few rows
df = pd.read_csv('reviews.csv')
print(df.head())
print(f'Shape of the data frame: {df_reviews.shape}')
df_reviews

                                               Title  \
0               5.0 out of 5 stars\nGreat bp machine   
1  5.0 out of 5 stars\nOutstanding Performance an...   
2                  4.0 out of 5 stars\nVery accurate   
3  4.0 out of 5 stars\nTakes a little practice to...   
4                              5.0 out of 5 stars\nL   

                                              Review  
0  Very accurate with my old machine which i thou...  
1  I've recently purchased the Konquest Automatic...  
2  I like that it is accurate, as measured agains...  
3  Given as a gift to my sister.  She likes the e...  
4  Simple to use a keeps track of my blood presur...  
Shape of the data frame: (2810, 2)


,Title,Review
0,5.0 out of 5 stars\nGreat bp machine,Very accurate with my old machine which i thou...
1,5.0 out of 5 stars\nOutstanding Performance an...,I've recently purchased the Konquest Automatic...
2,4.0 out of 5 stars\nVery accurate,"I like that it is accurate, as measured agains..."
3,4.0 out of 5 stars\nTakes a little practice to...,Given as a gift to my sister. She likes the e...
4,5.0 out of 5 stars\nL,Simple to use a keeps track of my blood presur...
...,...,...
2805,4.0 out of 5 stars\nIt works well,I use this new one and then try my old one and...
2806,5.0 out of 5 stars\nThe accuracy and softness ...,The case is challenging to put it in after eac...
2807,5.0 out of 5 stars\nEasy to use,The package arrived quickly and it was super e...
2808,5.0 out of 5 stars\nTerrific blood pressure mo...,Terrific blood pressure monitor. Super easy to...


In [2]:
df['Review'].isna().value_counts()

print(df.shape)
df = df.dropna()
print(df.shape)

(2810, 2)
(2810, 2)


In [3]:
# Load the regular expression library
import re
# Remove punctuation
df['Review_processed'] = \
df['Review'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['Review_processed'] = \
df['Review_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df['Review_processed'].head()

0    very accurate with my old machine which i thou...
1    i've recently purchased the konquest automatic...
2    i like that it is accurate as measured against...
3    given as a gift to my sister  she likes the ea...
4    simple to use a keeps track of my blood presur...
Name: Review_processed, dtype: object

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [4]:
# Write your code here
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]
data = df.Review_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['accurate', 'old', 'machine', 'thought', 'getting', 'old', 'giving', 'right', 'numbers', 'place', 'last', 'weekso', 'panic', 'bought', 'one', 'last', 'night', 'arrived', 'overnight', 'tested', 'spot', 'old', 'one', 'guess', 'rains', 'barometric', 'pressure', 'changes', 'time', 'year']


In [5]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 3), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 1), (29, 1)]


In [6]:
from pprint import pprint
# number of topics
num_topics = 15
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"blood" + 0.035*"pressure" + 0.035*"machine" + 0.029*"one" + '
  '0.022*"accurate" + 0.022*"reading" + 0.020*"monitor" + 0.019*"read" + '
  '0.019*"hospital" + 0.018*"problem"'),
 (1,
  '0.148*"challenging" + 0.148*"put" + 0.148*"case" + 0.047*"one" + '
  '0.033*"blood" + 0.031*"new" + 0.029*"keeps" + 0.029*"track" + '
  '0.029*"presure" + 0.029*"simple"'),
 (2,
  '0.049*"pressure" + 0.041*"blood" + 0.024*"device" + 0.024*"konquest" + '
  '0.020*"monitor" + 0.019*"automatic" + 0.017*"extremely" + 0.016*"highly" + '
  '0.015*"arm" + 0.015*"upper"'),
 (3,
  '0.067*"one" + 0.060*"pressure" + 0.051*"old" + 0.034*"last" + '
  '0.033*"machine" + 0.023*"blood" + 0.021*"accurate" + 0.021*"bought" + '
  '0.018*"giving" + 0.018*"numbers"'),
 (4,
  '0.057*"accuracy" + 0.055*"getting" + 0.054*"used" + 0.054*"get" + '
  '0.054*"given" + 0.054*"correctly" + 0.054*"likes" + 0.053*"ease" + '
  '0.053*"placed" + 0.053*"taking"'),
 (5,
  '0.109*"put" + 0.109*"challenging" + 0.109*"case" + 

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [7]:
# Write your code here
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):

    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(),
                                remove_stopwords,
                                strip_punctuation,
                                strip_short,
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)

    return text

# apply function to all reviews
df['Review_Text (Clean)'] = df['Review'].apply(lambda x: preprocess(x))
# preview of dataset
df.head()


,Title,Review,Review_processed,Review_Text (Clean)
0,5.0 out of 5 stars\nGreat bp machine,Very accurate with my old machine which i thou...,very accurate with my old machine which i thou...,"[accur, old, machin, thought, get, old, give, ..."
1,5.0 out of 5 stars\nOutstanding Performance an...,I've recently purchased the Konquest Automatic...,i've recently purchased the konquest automatic...,"[recent, purchas, konquest, automat, upper, ar..."
2,4.0 out of 5 stars\nVery accurate,"I like that it is accurate, as measured agains...",i like that it is accurate as measured against...,"[like, accur, measur, doctor’, blood, pressur,..."
3,4.0 out of 5 stars\nTakes a little practice to...,Given as a gift to my sister. She likes the e...,given as a gift to my sister she likes the ea...,"[given, gift, sister, like, eas, accuraci, but..."
4,5.0 out of 5 stars\nL,Simple to use a keeps track of my blood presur...,simple to use a keeps track of my blood presur...,"[simpl, us, keep, track, blood, presur, test, ..."


In [8]:
from gensim import corpora

# create a dictionary with the corpus
corpus = df['Review_Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Coherence score in topic modeling to measure how interpretable the topics are to humans.
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Review_Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.6611288571521587
Coherence score with 3 clusters: 0.7539901610789933
Coherence score with 4 clusters: 0.6836888251153056
Coherence score with 5 clusters: 0.7096807913188382
Coherence score with 6 clusters: 0.6846470257212052
Coherence score with 7 clusters: 0.5335644141804129
Coherence score with 8 clusters: 0.4205429174484512
Coherence score with 9 clusters: 0.47721732599020456
Coherence score with 10 clusters: 0.45667124230580464


In [9]:
# perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)

# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: -0.372*"pressur" + -0.342*"blood" + -0.312*"monitor" + -0.305*"read" + -0.213*"devic" + -0.213*"konquest" + -0.176*"us" + -0.144*"set" + -0.142*"arm" + -0.142*"automat".
Words in 1: 0.436*"machin" + 0.239*"old" + 0.237*"read" + 0.189*"accur" + 0.180*"hospit" + 0.178*"on" + 0.166*"bought" + 0.151*"rain" + 0.151*"week" + 0.149*"pressur".


In [10]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]


score1 = []
score2 = []
for doc in corpus_lsi:
    if len(doc) > 0 and len(doc[0]) > 1:
        score1.append(round(doc[0][1],2))
    if len(doc) > 1 and len(doc[1]) > 1:
        score2.append(round(doc[1][1],2))
# create data frame that shows scores assigned for both topics for each review
# Remove any reviews that were not processed by the LDA model
df = df.iloc[:len(score1)]

# Create a DataFrame with the review text and topic scores
df_topic = pd.DataFrame()
df_topic['Text'] = df['Review']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)

df_topic.head()

,Text,Topic 0 score,Topic 1 score,Topic
0,Very accurate with my old machine which i thou...,-2.81,4.53,1
1,I've recently purchased the Konquest Automatic...,-11.98,-2.86,1
2,"I like that it is accurate, as measured agains...",-1.61,1.18,1
3,Given as a gift to my sister. She likes the e...,-0.22,0.14,1
4,Simple to use a keeps track of my blood presur...,-0.55,0.08,1


In [12]:
# find a sample review from each topic
df_topic0 = df_topic[df_topic['Topic'] == 1]
df_topic1 = df_topic[df_topic['Topic']==1]
print('Sample text from topic 0:\n {}'.format(df_topic0.sample(1, random_state=2)['Text'].values))
print('Sample text from topic 1:\n {}'.format(df_topic1.sample(1, random_state=2)['Text'].values))

Sample text from topic 0:
 ["I bought this for my husband as he needs to monitor his blood pressure regularly. Surprisingly, this machine is accurate. We took it to the hospital to compare with their machine and this one, and it was only one point off. This machine read 116/75 and the hospital machine read 115/75. It was good to know how close the readings are to one another. The display is big and nice, and it makes a sound once it's done reading. I'd definitely recommend."]
Sample text from topic 1:
 ["I bought this for my husband as he needs to monitor his blood pressure regularly. Surprisingly, this machine is accurate. We took it to the hospital to compare with their machine and this one, and it was only one point off. This machine read 116/75 and the hospital machine read 115/75. It was good to know how close the readings are to one another. The display is big and nice, and it makes a sound once it's done reading. I'd definitely recommend."]


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [13]:
# Write your code here

!pip install pyLDAvis

  Using cached numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.1 which is incompatible.


In [14]:
import nltk
nltk.download('all')
!pip install preprocess
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [15]:
import pyLDAvis
pyLDAvis.enable_notebook()
top = 10
topic_to_topwords = {}
for j, topic in enumerate(df['Review_Text (Clean)'].tolist()):
    top1 = np.argsort(topic)[::-1][:top]
    msg = 'Topic %i '  % j
    tp_wrd = [s.strip()[:35] for s in df['Review_Text (Clean)'][i] for i in top1]
    msg += ' has top words '.join(tp_wrd)
    print(msg)
    topic_to_topwords[j] = tp_wrd


Topic 0 accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words ol

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Topic 2714 accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words accur has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words machin has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words thought has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words get has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words old has top words give has top words give has top words give has top w

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [16]:
# Write your code here

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.Review.apply(clean_text)
df.head()




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:21: DeprecationWarning: invalid escape sequence '\('
<>:21: DeprecationWarning: invalid escape sequence '\('
<ipython-input-16-07888fe9d326>:21: DeprecationWarning: invalid escape sequence '\('
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Title,Review,Review_processed,Review_Text (Clean),clean_text
0,5.0 out of 5 stars\nGreat bp machine,Very accurate with my old machine which i thou...,very accurate with my old machine which i thou...,"[accur, old, machin, thought, get, old, give, ...",accurate old machine thought getting old givin...
1,5.0 out of 5 stars\nOutstanding Performance an...,I've recently purchased the Konquest Automatic...,i've recently purchased the konquest automatic...,"[recent, purchas, konquest, automat, upper, ar...",'ve recently purchased konquest automatic uppe...
2,4.0 out of 5 stars\nVery accurate,"I like that it is accurate, as measured agains...",i like that it is accurate as measured against...,"[like, accur, measur, doctor’, blood, pressur,...",like accurate measured doctor blood pressure c...
3,4.0 out of 5 stars\nTakes a little practice to...,Given as a gift to my sister. She likes the e...,given as a gift to my sister she likes the ea...,"[given, gift, sister, like, eas, accuraci, but...",given gift sister likes ease using accuracy ta...
4,5.0 out of 5 stars\nL,Simple to use a keeps track of my blood presur...,simple to use a keeps track of my blood presur...,"[simpl, us, keep, track, blood, presur, test, ...",simple use keeps track blood presure test resu...


In [16]:
! pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (

In [17]:
from bertopic import BERTopic

reviews= df.clean_text.to_list()
timestamp = df.Review.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(reviews)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
topic_model.visualize_barchart(top_n_topics=12, n_words = 10, width = 350, height = 350)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

All three algorithms have similar documents at the top and bottom of the 100 patent document categories.

1) LSA outperforms LTA and BERT because it can decompose the original matrix using an information retrieval method while retaining key topics. LDA works on the assumption that documents are generated from a variety of topics, which are then output as words using a probability distribution.

2)The disadvantage of LDA2Vec is that the context/paragraph vectors resemble standard word vectors, making them less interpretable to the  output of LDA.

3) The lda2vec model takes the paragraph vector approach a step further by working with document-sized text fragments and decomposing the document vectors into two distinct components.


4)A document vector is decomposed into a document weight vector and a topic matrix in the same way that the LDA model is. The percentage of the different topics is represented by the document weight vector, whereas the topic matrix is made up of the different topic vectors.